# Sweep functions
This notebook shows how to make basic sweeps. [mesoscoPy](https://github.com/julienbarrier/mesoscopy) integrates function to perform the following:
* **fast sweep**: sweep a parameter to a target value, without measuring.
* **1D sweep**: sweep a parameter along an array. included in a measurement object.
* **time sweep**: sweep time, while measuring other parameter. included in a measurement object.
* **repeated 1D sweep**: sweep a parameter along an array, and repeat a certain number of time. included in a measurement object.
* **2D sweep**: sweep a parameter along an array (outer sweep), and for each value, sweep another parameter (inner sweep). included in a measurement object.


We assume here to be working with a Keithley 2600, lock-ins Zurich Instruments MFLI and an Oxford Triton

## Imports

In [ ]:
%matplotlib inline

from qcodes import initialise_or_create_database_at, load_or_create_experiment
from mesoscopy.instrument.station import init_station
from mesoscopy.instrument.keithley import initialise_keithley
from mesoscopy.instrument.lockin import initialise_lockin, enable_DC, disable_DC
from mesoscopy.experiment.characterise_contacts import contact_IV, test_gate, twoprobe_contacts
from mesoscopy.measurement.array import generate_1D_sweep_array

## Set up station and database
### set up measurement station

we use the function `init_station` from `mesoscopy.instrument.station`. It is composed of the following arguments:
* `keithley_addr`: Visa address (string)
* `triton_addr`: IP address (stringg)
* `MFLI_num`: number of MFLI (string)
* `current_range`: factor for the input current. the first MFLI sources an AC voltage. We convert this to a current source by adding a resistor in series. `current_range` correspnods to the current sourced with a 1V AC excitation. Optional value.

Example:

In [ ]:
station = init_station("TCPIP::192.168.0.13::inst0::INSTR", 
                       # we are loading a keithley 2614B on ip 192.168.0.13
                       "192.168.0.2",  # we are loading the Oxford triton
                       "4406","4330", "3550",  # we are loading 3 MFLIs: MF4406, MF4330, MF3550
                       current_range=1e-9)

### initialise instruments
The keithley can be initialised with voltage compliants limits (`limits_v` in Volts) and maximum sweeping rate (`max_rate` in Volt/sec). These take tuples (or sequences) to account for multiple channels.
The maximum sweeping rate is coded in [mesoscoPy](https://github.com/julienbarrier/mesoscopy)'s functions. It is not shared with the instrument.

In [ ]:
initialise_keithley(
    station,  # the station we initialised
    limits_v=[20, 60],  # limits for the gates, controlled by smua and smub
    max_rate=[0.12, 0.12]  # maximum sweeping rates
    )

The lockins can be initialised by setting up the amplitude (`ampl` in Volts) and the frequency (`freq`, in Hz):

In [ ]:
initialise_lockin(
    station,
    freq=127,  # lock-in frequency in Hz
    ampl=1  # 1V output
)

### Name and initialise database

In [ ]:
initialise_or_create_database_at('../database.db')
exp = load_or_create_experiment(
    experiment_name='test sweeps',
    sample_name='no_sample'
)

## 1D sweep
We are going to perform a gate sweep. For the following, we shall connect the Keithley's channel A to the gate of interest, and float everything else.
### initialise the sweep

In [ ]:
# first, we sweep the gate to 40V in 0.1V steps (optional)
fastsweep(
    40,  # target value
    station.keithley.smub.volt,  # instrument parameter to sweep
    step=.1  # step size (Optional, useful if actions or control are registered)
    actions=None,  # actions to perform in between each steps
    control=None,  # callable, called at each step. if true, breaks the sweep
    lbar=True  # display aprogress bar
)

# second, we initialise an array along which we are going to perform the sweep
array = generate_1D_sweep_array(
    40,  # init
    -40, # stop
    step=0.2 # step size. alternatively, one can set up a number of steps with ``num``
)

### sweep a parameter and record other values

In [ ]:
sweep1d(
    station.keithley.smub.volt,  # parameter to sweep
    array,  # array to sweep along
    1, # settling time (sec)
    station.mf4406.demods[0].sample,  # parameters to be measured during the sweep
    station.mf4330.demods[0].sample,
    station.mf3550.demods[0].sample,
    station.keithley.smua.volt,
    station.keithley.smua.curr,
    station.keithley.smub.curr,
    station.triton.T8,
    station.triton.Bz,
    exp=exp,  # experiment object
    measurement_name='Vbg sweep -40V::40V',  # measurement name
    do_plot=False,  # plotting the sweep?
    use_threads=True,  # use threads to parallelise records from instrument
    enter_actions=None,  # actions to be performed before the sweep starts
    exit_actions=None,  # actions to be performed after the sweep ends
    additional_setpoints=tuple()  # sequence of additional setponits
)

### time sweep
this is very similar to a 1D sweep, with the exception that it doesn't take a parameter to sweep:

In [ ]:
sweeptime(
    1,  # settling time (sec)
    100,  # duration of the sweep (sec)
    station.mf4406.demods[0].sample,  # parameters to be measured during the sweep
    station.mf4330.demods[0].sample,
    station.mf3550.demods[0].sample,
    exp=exp,  # experiment object
    measurement_name='time sweep to 100s',  # measurement name
    use_threads=True,  # use threads to parallelise records from instrument
    enter_actions=None,  # actions to be performed before the sweep starts
    exit_actions=None,  # actions to be performed after the sweep ends
    additional_setpoints=tuple()  # sequence of additional setponits
)

## 2D sweeps
There are two kinds of 2D sweeps. eitheir we repeat a 1D sweep with actions in between each of them, or we can sweep a parameter for each repetition of the nested 1D sweep.

### repeat 1D sweep
the main goal of this function is to test the reproducibility of a sweep, without restarting a 1D sweep multiple times. alternatively, one can add outer_enter_actions to vary different parameters of the experiment.

In [1]:
# preliminary:
array = generate_1D_sweep_array(20,-20, num=40)
fastsweep(20, station.keithley.smub.volt)
# this fastweep is optional: sweep1d_repead already includes a fastsweep for safety reasons

sweep1d_repeat(
    station.keithley.smub.volt,  # parameter to sweep
    array,  # array to sweep along
    1, # settling time (sec)
    station.mf4406.demods[0].sample,  # parameters to be measured during the sweep
    station.mf4330.demods[0].sample,
    station.mf3550.demods[0].sample,
    station.keithley.smua.volt,
    station.keithley.smua.curr,
    station.keithley.smub.curr,
    station.triton.T8,
    station.triton.Bz,
    10,  # number of repeats,
    0.5,  # time to wait between steps
    exp=exp,  # experiment object
    measurement_name='Vbg sweep 20V::-20V - repeat',  # measurement name
    do_plot=False,  # plotting the sweep?
    use_threads=True,  # use threads to parallelise records from instrument
    outer_enter_actions=None,  # actions to be performed before the first sweep starts
    outer_exit_actions=None,  # actions to be performed after the final sweep ends
    inner_enter_actions=None,  # actions to be performed before an individual sweep
    inner_exit_actions=None,  # actions to be performed after an individual sweep
    measure_retrace=False,  # bool: do we want to record the retrace measurement?
    additional_setpoints=tuple()  # sequence of additional setponits
)

NameError: name 'generate_1D_sweep_array' is not defined

### 2D sweep
this is useful to make a map while varying two parameters.
For example, we are going to measure IV curves (and dV/dI) while sweeping the gate

First, we enable DC measurements on ZI lock-ins:

In [ ]:
enable_DC(station)  # we remove the AC filter on all ZI lock-ins, to be able to measure IV curves.
# Additionally, this enables a demodulator at frequency 0 on all lock-ins registered in the station.

Second, we initialise our measurement:

In [ ]:
exp = load_or_create_experiment(
    experiment_name='IV gate dependence',
    sample_name='no_sample'
)

xarray = generate_1D_sweep_array(.5,-.5, num=151)
yarray = generate_1D_sweep_array(20,-20, step=0.5)

fastsweep(.5,station.mf4406.auxouts[0].offset)
fastsweep(20, station.keithley.smub.volt)

Finaly, we start trhe 2D sweep

In [ ]:
sweep2d(
    station.mf4406.auxouts[0].offset, # parameter to sweep (fast sweep)
    array,  # array to sweep along
    1, # settling time (sec)
    station.keithley.smub.volt,  # parameter to sweep (slow sweep)
    yarray,  # array to sweep along
    .5,  # waiting time before the fast sweep starts (y-settling time)
    station.mf4406.demods[0].sample,  # parameters to be measured during the sweep
    station.mf4406.demods[2].sample,  # here demods[2] corresponds to DC voltage
    station.mf4330.demods[0].sample,
    station.mf4330.demods[2].sample,
    station.mf3550.demods[0].sample,
    station.mf3550.demods[2].sample,
    station.keithley.smua.volt,
    station.keithley.smua.curr,
    station.keithley.smub.curr,
    station.triton.T8,
    station.triton.Bz,
    exp=exp,  # experiment object
    measurement_name='IV curves for Vbg sweep 20V::-20V',  # measurement name
    do_plot=False,  # plotting the sweep?
    use_threads=True,  # use threads to parallelise records from instrument
    outer_enter_actions=None,  # actions to be performed before the first sweep starts
    outer_exit_actions=None,  # actions to be performed after the final sweep ends
    inner_enter_actions=None,  # actions to be performed before an individual sweep
    inner_exit_actions=None,  # actions to be performed after an individual sweep
    measure_retrace=False,  # bool: do we want to record the retrace measurement?
    additional_setpoints=tuple()  # sequence of additional setponits
)